In [10]:
import itertools
import random
import ast
import pandas as pd
import numpy as np
from collections import defaultdict

In [11]:
# Load in all metric patterns from combos 5, 6, and 7
metric_5 = pd.read_csv('stim_conditions/combo_chart_5.csv')
metric_6 = pd.read_csv('stim_conditions/combo_chart_6.csv')
metric_7 = pd.read_csv('stim_conditions/combo_chart_7.csv')
metric_patterns = pd.concat([metric_5, metric_6, metric_7], axis=0)
simple_patterns = [[int(x) for x in ast.literal_eval(pattern)] for pattern in metric_patterns['simple'].dropna().tolist()]
complex_patterns = [[int(x) for x in ast.literal_eval(pattern)] for pattern in metric_patterns['complex'].dropna().tolist()]
simple_silence_patterns = [[int(x) for x in ast.literal_eval(pattern)] for pattern in metric_patterns['simple_silence'].dropna().tolist()]
complex_silence_patterns = [[int(x) for x in ast.literal_eval(pattern)] for pattern in metric_patterns['complex_silence'].dropna().tolist()]

In [12]:
# Keep patterns that have at least 3 unique metrics
simple_patterns = [pattern for pattern in simple_patterns if len(set(pattern)) >= 2]
complex_patterns = [pattern for pattern in complex_patterns if len(set(pattern)) >= 2]

In [13]:
# Function to generate all unique combinations of four words without repetitions
def generate_unique_combinations(words):
    return list(itertools.permutations(words, 4))

def arrange_syllables(unique_combinations, previous_combination, used_combinations, cycle_limit):
    random.shuffle(unique_combinations)  # Shuffle to ensure a random selection each time
    for combo in unique_combinations:
        # Check if we have used all combinations in this cycle
        if len(used_combinations) >= cycle_limit:
            used_combinations.clear()  # Reset for a new cycle

        # Ensure new combo is not the same as previous and does not start with previous combo's last element
        if combo != previous_combination \
           and (not previous_combination or combo[0] != previous_combination[-1]) \
           and combo not in used_combinations:
            return combo
    return None  # This should ideally not occur

def interweave(syllables, silence_pattern):
    # Ensure syllables are broken into individual letters
    flattened_syllables = [letter for word in syllables for letter in word]
    
    # Create the interweaved sequence
    interweaved = []
    for letter, silence in zip(flattened_syllables, silence_pattern):
        interweaved.append(letter)
        interweaved.append(silence)
    
    return interweaved

def pick_pattern_template_and_syllables(condition):
    # Choose appropriate patterns based on the condition
    patterns = simple_patterns if condition == 'simple' else complex_patterns
    silence_patterns = simple_silence_patterns if condition == 'simple' else complex_silence_patterns

    # Flatten the patterns if they are lists of lists
    patterns = [item for sublist in patterns for item in sublist]
    silence_patterns = [item for sublist in silence_patterns for item in sublist]

    # Initialize the dictionary to keep track of metric values
    metric_values = {1: 0, 2: 0, 3: 0, 4: 0}
    selected_patterns = []
    last_metric_value = None
    
    for pattern in patterns:
        # Convert pattern to string if it's an integer
        pattern = str(pattern) if isinstance(pattern, int) else pattern

        # Calculate the metric value count for the current pattern
        metric_value_count = {1: pattern.count('1'), 2: pattern.count('2'), 3: pattern.count('3'), 4: pattern.count('4')}

        # Check if the first metric value in the pattern is the same as the last used metric value
        if pattern[0] == last_metric_value:
            continue

        # Check if adding the current pattern would maintain the balance of metric values
        if all(metric_values[i] + metric_value_count[i] <= len(selected_patterns) // 4 + 1 for i in range(1, 5)):
            # If the balance is maintained, add the pattern to the selected patterns and update the dictionary
            selected_patterns.append(pattern)
            for i in range(1, 5):
                metric_values[i] += metric_value_count[i]
            
            # Update the last used metric value
            last_metric_value = pattern[-1]
        
    # Combine selected patterns into one string each
    pattern = ''.join([str(x) for x in selected_patterns])
    # print(pattern)
    return pattern

In [17]:
words = ['ABC', 'DEF', 'GHI', 'JKL']
unique_combinations = generate_unique_combinations(words)
arranged_syllables = []
previous_combination = None
used_combinations = set()

cycle_limit = len(unique_combinations)
for _ in range(48):  # We aim for 48 sets
    new_set = arrange_syllables(unique_combinations, previous_combination, used_combinations, cycle_limit)
    if new_set:
        arranged_syllables.append(new_set)
        used_combinations.add(new_set)
        previous_combination = new_set
        # flatten the list of lists
flattened_syllables = [letter for word in arranged_syllables for letter in word]
print('flattened_syllables:', flattened_syllables)

flattened_syllables: ['ABC', 'JKL', 'GHI', 'DEF', 'GHI', 'ABC', 'JKL', 'DEF', 'JKL', 'GHI', 'DEF', 'ABC', 'JKL', 'ABC', 'DEF', 'GHI', 'DEF', 'JKL', 'ABC', 'GHI', 'ABC', 'GHI', 'JKL', 'DEF', 'JKL', 'DEF', 'ABC', 'GHI', 'ABC', 'DEF', 'GHI', 'JKL', 'DEF', 'GHI', 'JKL', 'ABC', 'DEF', 'ABC', 'GHI', 'JKL', 'DEF', 'ABC', 'JKL', 'GHI', 'ABC', 'JKL', 'DEF', 'GHI', 'ABC', 'GHI', 'DEF', 'JKL', 'GHI', 'DEF', 'ABC', 'JKL', 'GHI', 'DEF', 'JKL', 'ABC', 'GHI', 'JKL', 'DEF', 'ABC', 'JKL', 'GHI', 'ABC', 'DEF', 'GHI', 'ABC', 'DEF', 'JKL', 'ABC', 'DEF', 'JKL', 'GHI', 'JKL', 'ABC', 'GHI', 'DEF', 'JKL', 'DEF', 'GHI', 'ABC', 'DEF', 'JKL', 'GHI', 'ABC', 'DEF', 'GHI', 'ABC', 'JKL', 'GHI', 'JKL', 'ABC', 'DEF', 'ABC', 'DEF', 'JKL', 'GHI', 'ABC', 'JKL', 'DEF', 'GHI', 'ABC', 'JKL', 'GHI', 'DEF', 'JKL', 'DEF', 'ABC', 'GHI', 'JKL', 'GHI', 'ABC', 'DEF', 'JKL', 'ABC', 'DEF', 'GHI', 'ABC', 'GHI', 'DEF', 'JKL', 'GHI', 'ABC', 'JKL', 'DEF', 'GHI', 'DEF', 'JKL', 'ABC', 'JKL', 'ABC', 'GHI', 'DEF', 'ABC', 'DEF', 'GHI', 'JKL'

In [18]:
interweaved_sequence = interweave(flattened_syllables, pick_pattern_template_and_syllables('simple'))
valid_pairs = [
    ('A', 'B'), ('B', 'C'), ('C', 'D'), ('C', 'G'), ('C', 'J'),
    ('D', 'E'), ('E', 'F'), ('F', 'G'), ('F', 'A'), ('F', 'J'),
    ('G', 'H'), ('H', 'I'), ('I', 'J'), ('I', 'A'), ('I', 'D'),
    ('J', 'K'), ('K', 'L'), ('L', 'A'), ('L', 'D'), ('L', 'G')
]
valid_pair_set = set(valid_pairs)
pair_transition_freq = defaultdict(lambda: defaultdict(int))

# Regular counting for all but the last pair
for i in range(0, len(interweaved_sequence) - 2, 2):  # Step through individual pairs
    current_pair = (interweaved_sequence[i], interweaved_sequence[i + 2])
    # Only count if current pair is valid
    if current_pair in valid_pair_set:
        silence_value = interweaved_sequence[i + 1]
        pair_transition_freq[current_pair][silence_value] += 1

# This checks if such a pair (last-first) is a valid pair to be counted
if len(interweaved_sequence) > 2:  # Ensure there are enough elements for a wrap-around
    last_pair = (interweaved_sequence[-2], interweaved_sequence[0])
    last_silence = interweaved_sequence[-1]
    if last_pair in valid_pair_set:
        pair_transition_freq[last_pair][last_silence] += 1

# Convert the frequency table to a DataFrame for analysis and display
df_pair_transitions = pd.DataFrame(pair_transition_freq).T.fillna(0).astype(int)
df_pair_transitions['Total'] = df_pair_transitions.sum(axis=1)
df_pair_transitions.sort_values('Total', ascending=False, inplace=True)
df_pair_transitions = df_pair_transitions.sort_index(axis=1)  # Sorting for better visualization and consistency

# Display the refined frequency table
print("Simple Transitional Frequencies"), print()
display(df_pair_transitions)


Simple Transitional Frequencies



,,1,2,3,4,Total
A,B,15,16,8,9,48
G,H,14,8,12,14,48
B,C,8,13,10,17,48
D,E,10,12,12,14,48
H,I,9,14,15,10,48
E,F,16,17,12,3,48
K,L,12,13,16,7,48
J,K,13,10,13,12,48
I,A,3,5,9,2,19
F,J,2,3,4,9,18


In [9]:
import itertools
import random
from collections import defaultdict
import pandas as pd

interweaved_sequence = 'J4K2L2A1B4C4D1E2F2G3H4I4J3K1L1A1B3C3G3H1I1D1E4F4J3K2L2D4E1F1A4B1C1G4H2I2A4B2C2G4H1I1J1K3L3D2E3F3A3B1C1J1K2L2D4E3F3G3H2I2A4B4C4J3K2L2G4H4I4D3E4F4A2B3C3D2E3F3G1H4I4J1K1L1A1B2C2D1E3F3J4K1L1G1H2I2A4B3C3G3H1I1D4E1F1J1K1L1D2E4F4G2H1I1J4K3L3A4B2C2D3E2F2J1K2L2A1B2C2G2H3I3D1E3F3J2K1L1G3H4I4A2B1C1D1E4F4A3B2C2G1H4I4J2K2L2D2E1F1A1B4C4J4K2L2G4H3I3D4E1F1G4H3I3A1B1C1J3K4L4G2H2I2A4B4C4D2E1F1J2K4L4G4H1I1A2B4C4J2K3L3D2E3F3G1H1I1D4E4F4A3B4C4J2K3L3G1H1I1D3E2F2J4K4L4A2B3C3G2H4I4J2K2L2A3B1C1D4E2F2G1H3I3J4K1L1D3E1F1A4B1C1J3K3L3D2E4F4G3H2I2A4B4C4J1K1L1G4H1I1A2B3C3D3E4F4J4K1L1G2H4I4D2E4F4A2B3C3J3K1L1A4B2C2D3E2F2G3H4I4J2K3L3A3B2C2G2H3I3D3E1F1J1K2L2D1E2F2A3B3C3G3H2I2A1B2C2G3H3I3J2K3L3D2E2F2A2B4C4J1K4L4D1E1F1G1H4I4A4B2C2J1K1L1G2H4I4D3E2F2A1B4C4D4E3F3G3H1I1J3K4L4A1B2C2D2E4F4J2K1L1G2H2I2A3B1C1G3H3I3D3E3F3J3K1L1D1E4F4G2H4I4J4K2L2A2B4C4D3E1F1J3K4L4A2B1C1G1H2I2D1E3F3J1K4L4G4H1I1A4B3C3D4E2F2A4B3C3G1H1I1J2K4L4D1E2F2A2B1C1J4K2L2G4H3I3D1E3F3G1H2I2A1B1C1J3K3L3G1H3I3A1B2C2D4E3F3J1K4L4G4H2I2A3B4C4J3K4L4D4E2F2G1H3I3D3E3F3A2B1C1J4K3L3G3H3I3D4E2F2J4K4L4A3B3C3G4H2I2J4K3L3A2B4C4D1E1F1G2H3I3J2K4L4D4E4F4A3B1C1J3K3L3D2E1F1G4H4I4A3B3C3J1K3L3G2H2I2A1B2C2D3E1F1J2K2L2G2H1I1D2E4F4A3B3C3'
valid_pairs = [
    ('A', 'B'), ('B', 'C'), ('C', 'D'), ('C', 'G'), ('C', 'J'),
    ('D', 'E'), ('E', 'F'), ('F', 'G'), ('F', 'A'), ('F', 'J'),
    ('G', 'H'), ('H', 'I'), ('I', 'J'), ('I', 'A'), ('I', 'D'),
    ('J', 'K'), ('K', 'L'), ('L', 'A'), ('L', 'D'), ('L', 'G')
]
valid_pair_set = set(valid_pairs)
pair_transition_freq = defaultdict(lambda: defaultdict(int))
# Regular counting for all but the last pair
for i in range(0, len(interweaved_sequence) - 2, 2):  # Step through individual pairs
    current_pair = (interweaved_sequence[i], interweaved_sequence[i + 2])
    # Only count if current pair is valid
    if current_pair in valid_pair_set:
        silence_value = interweaved_sequence[i + 1]
        pair_transition_freq[current_pair][silence_value] += 1


# This checks if such a pair (last-first) is a valid pair to be counted
if len(interweaved_sequence) > 2:  # Ensure there are enough elements for a wrap-around
    last_pair = (interweaved_sequence[-2], interweaved_sequence[0])
    last_silence = interweaved_sequence[-1]
    if last_pair in valid_pair_set:
        pair_transition_freq[last_pair][last_silence] += 1

# Convert the frequency table to a DataFrame for analysis and display
df_pair_transitions = pd.DataFrame(pair_transition_freq).T.fillna(0).astype(int)
df_pair_transitions['Total'] = df_pair_transitions.sum(axis=1)
df_pair_transitions.sort_values('Total', ascending=False, inplace=True)
df_pair_transitions = df_pair_transitions.sort_index(axis=1)  # Sorting for better visualization and consistency

# Display the refined frequency table
print("Complex Transitional Frequencies"), print()
display(df_pair_transitions)


Complex Transitional Frequencies



,,1,2,3,4,Total
J,K,12,12,12,12,48
D,E,12,12,12,12,48
K,L,12,12,12,12,48
G,H,12,12,12,12,48
E,F,12,12,12,12,48
H,I,12,12,12,12,48
B,C,12,12,12,12,48
A,B,12,12,12,12,48
I,A,3,10,2,3,18
C,J,7,1,3,7,18


In [176]:
# from collections import defaultdict
# import pandas as pd

# # Assuming the functions interweave and pick_pattern_template_and_syllables are defined elsewhere

# valid_pairs = [
#     ('A', 'B'), ('B', 'C'), ('C', 'D'), ('C', 'G'), ('C', 'J'),
#     ('D', 'E'), ('E', 'F'), ('F', 'G'), ('F', 'A'), ('F', 'J'),
#     ('G', 'H'), ('H', 'I'), ('I', 'J'), ('I', 'A'), ('I', 'D'),
#     ('J', 'K'), ('K', 'L'), ('L', 'A'), ('L', 'D'), ('L', 'G')
# ]
# valid_pair_set = set(valid_pairs)

# perfect_distribution = False
# desired_count = 12

# while not perfect_distribution:
#     interweaved_sequence = interweave(flattened_syllables, pick_pattern_template_and_syllables('complex'))
#     pair_transition_freq = defaultdict(lambda: defaultdict(int))

#     for i in range(0, len(interweaved_sequence) - 2, 2):
#         current_pair = (interweaved_sequence[i], interweaved_sequence[i + 2])
#         if current_pair in valid_pair_set:
#             silence_value = interweaved_sequence[i + 1]
#             pair_transition_freq[current_pair][silence_value] += 1

#     if len(interweaved_sequence) > 2:
#         last_pair = (interweaved_sequence[-2], interweaved_sequence[0])
#         last_silence = interweaved_sequence[-1]
#         if last_pair in valid_pair_set:
#             pair_transition_freq[last_pair][last_silence] += 1

#     perfect_distribution = all(
#         all(count == desired_count for count in transition_freq.values())
#         for transition_freq in pair_transition_freq.values()
#     )

# df_pair_transitions = pd.DataFrame(pair_transition_freq).T.fillna(0).astype(int)
# df_pair_transitions['Total'] = df_pair_transitions.sum(axis=1)
# df_pair_transitions.sort_values('Total', ascending=False, inplace=True)
# df_pair_transitions = df_pair_transitions.sort_index(axis=1)

# display(df_pair_transitions)